# Testing Semtiment Analysis for the Detection of Ingroup and Outgroup
---

Over three tests, IBM's Watson and TextBlob were used to determine whether sentiment could detect ingroups and outgroups of a text, scores related to seed words of the group schema and elevation and otherising statements. 

For detecting the ingroup and outgroup of a text the dataset was processed to produce list of named entities for each orator. These lists were then processed by TextBlob and Watson to produce the scored results. The same process was used for scoring the seed words with a list of named concepts being used. 

Finally for detecting elevation and otherising statements, the statements containing both a named entity and concept were extracted from the dataset and scored. Sentences classed as positive or negative were classed as elevation or otherising respectively. 

## Benchmark Data

A successful test will identify each of the following entities as either ingroup or outgroup.

In [2]:
import os
import json
import pandas as pd

filepath = "C:/Users/Steve/OneDrive - University of Southampton/CulturalViolence/KnowledgeBases/Data/"

with open(os.path.join(filepath, "groups_benchmark.json"), "r") as f:
    groups = json.load(f)
    
keys = list(groups.keys())
print(keys)

frames = []
for value in groups.values():
    frames.append(pd.DataFrame(dict([ (k, pd.Series(v)) for k, v in value.items() ]), index = None).fillna(""))

display(pd.concat(frames , keys = keys))

['bush', 'binladen']


ingroup                        outgroup
bush     0                   america                        al qaeda
         1           american people                  taliban regime
         2                 americans                         taliban
         3             united states          egyptian islamic jihad
         4  united states of America  islamic movement of uzbekistan
         5       my fellow americans                                
         6          fellow americans                                
binladen 0           people of islam      zionist-crusaders alliance
         1             islamic world              american crusaders
         2            ummah of islam       american zionist alliance
         3                   muslims       american-israeli alliance
         4             muslim people                    saudi regime
         5             muslim nation                  american enemy
         6                                         zionist-crusaders

## Set up the pipeline

In [3]:
import os
import sys
import platform
import json
import pandas as pd
import datetime
import tqdm
import spacy
from spacy.tokens import Span
from spacy.pipeline import merge_entities
from spacy.matcher import Matcher
from spacy import displacy

from VPipeLibrary.custpipe import EntityMatcher


pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.max_rows", 2000)

spacy.info()
print('============================== Info about python ==============================')
print('python version: ', platform.sys.version)
print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

ModuleNotFoundError: No module named 'VPipeLibrary'

In [854]:
print('=========================== Loading Language Models ===========================')
model = 'en_core_web_md'

print('loading', model)
nlp = spacy.load(model)

print('loaded', model)
      
print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

=========================== Loading Language Models ===========================
loading en_core_web_md
loaded en_core_web_md
completed at: Dec 19 2019 18:20:29


In [870]:
class NamedConcept(object):
    
    def __init__(self, nlp):
    
        self.matcher = Matcher(nlp.vocab)
        
        with open(r'C:\Users\Steve\Documents\CulturalViolence\KnowledgeBases\group_typology.json', 'r') as fp:
            self.named_concept_rules = json.load(fp)
            
        for entry in self.named_concept_rules.values():        
            for pattern in entry.values():
                for subcat, terms in pattern.items():
                    self.matcher.add(subcat, None, [{"LEMMA" : {"IN" : terms}}])
                    
    def __call__(self, doc):

        matches = self.matcher(doc)
        spans = []  # keep the spans for later so we can merge them afterwards

        for match_id, start, end in matches:
            
            ## gather up noun phrases
            
            
            concept = Span(doc, start, end, label=doc.vocab.strings[match_id])
            

            if "the" in [word.lower_ for word in list(doc[start].lefts)]:
                
                Span(doc, start - doc[start].n_lefts, end + doc[start].n_rights, label = doc.vocab.strings[match_id])
                #print(concept, '=>', concept.label_)
            
#             elif doc[start].dep_ in ["poss", "compound"] and end != len(doc):
#                 try:
#                     concept = Span(doc, start, list(doc[start + 1].rights)[-1].i + 1, label=doc.vocab.strings[match_id])
#                     print(concept, '=>', concept.label_)
#                 except:
#                     continue
            
            elif doc[start - 1].dep_ in ["amod", "compound"] and start != 0: 
                if doc[start -1].ent_type_:
                    concept = Span(doc, start - 1, end, label=doc[start -1].ent_type_)
                else:
                    concept = Span(doc, start - 1, end, label=doc.vocab.strings[match_id])
                #print(concept, '=>', concept.label_)

            elif doc[start - 1].pos_ in ["NOUN", "PROPN"] and start != 0:
                concept = Span(doc, start - 1, end, label=doc.vocab.strings[match_id])
                #print(concept, '=>', concept.label_)
                
            elif doc[start + 1].pos_ in ["NOUN", "PROPN"] and end != len(doc):
                concept = Span(doc, start, end + 1, label=doc.vocab.strings[match_id])
                #print(concept, '=>', concept.label_)

#             elif doc[start - 2].dep_ in ["nsubj", "amod"] and doc[start].dep_ in ["pobj"] and start != 0:
#                 concept = Span(doc, start - 2, end, label=doc.vocab.strings[match_id])
#                 #print(concept, '=>', concept.label_)
                
            elif doc[start].dep_ in ["nsubj", "csubj", "pobj"] and end != len(doc):
                if doc[start + 1].dep_ in ["prep"]: 
                    try:
                        concept = Span(doc, start, list(doc[start + 1].rights)[-1].i + 1, label=doc.vocab.strings[match_id])
                        #print(concept, '=>', concept.label_)
                    except:
                        continue
                        
            doc.ents = spacy.util.filter_spans(list(doc.ents) + [concept])
            
            spans.append(concept)
        
        with doc.retokenize() as retokenizer:
            # Iterate over all spans and merge them into one token. This is done
            # after setting the entities – otherwise, it would cause mismatched
            # indices!
            for span in spacy.util.filter_spans(doc.ents):
                retokenizer.merge(span)
            
        return doc  # don't forget to return the Doc!

from spacy.pipeline import EntityRuler

# remove all pipeline components
for pipe in nlp.pipe_names:
    if pipe not in ['tagger', "parser", "ner"]:
        nlp.remove_pipe(pipe)

#add entity ruler to the pipe

with open(r'C:\Users\Steve\Documents\CulturalViolence\KnowledgeBases\group_typology.json', 'r') as fp:
    group_typology = json.load(fp)

ruler = EntityRuler(nlp)
patterns = []
for entry in group_typology.values():        
    for pattern in entry.values():
        for subcat, terms in pattern.items():
            patterns.append({"label" : subcat, "pattern" : [{"LEMMA" : {"IN" : terms}}]})

# add new pipe components
ruler.add_patterns(patterns)            
#nlp.add_pipe(ruler)
nlp.add_pipe(EntityMatcher(nlp), before = "ner")
nlp.add_pipe(NamedConcept(nlp), after = "ner")

nlp.add_pipe(merge_entities)

print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

completed at: Dec 19 2019 19:14:47


In [5]:
##Set up filepaths


filepath = 'C:/Users/Steve/OneDrive - University of Southampton/CulturalViolence/KnowledgeBases/Speeches/'

binladenpath = os.path.join(filepath, 'Osama bin Laden/')
bushpath = os.path.join(filepath, 'George Bush/')

filepath = "C:/Users/Steve/OneDrive - University of Southampton/CulturalViolence/KnowledgeBases/data/"
resultspath = "C:/Users/Steve/OneDrive - University of Southampton/CulturalViolence/KnowledgeBases/Experiment 4 - Testing Sentiment Analysis to Detect Ingroup and Outgroup/"

print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

completed at: Feb 13 2020 15:36:42


### IBM Watson

(insert information about IBM Watson)

In [345]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, SentimentOptions

apikey = ''
url = ''

authenticator = IAMAuthenticator(apikey)
service = NaturalLanguageUnderstandingV1(version='2019-07-12', authenticator=authenticator)
service.set_service_url(url)
    
# response = service.analyze(
#     text="The evidence we have gathered all points to a collection of loosely affiliated terrorist organizations known as al-Qa\'eda.",
#     features=Features(sentiment=SentimentOptions()   
#                      )).get_result()
# #entities=EntitiesOptions(emotion=True, sentiment=True, limit=2)
# #keywords=KeywordsOptions(emotion=True, sentiment=True, limit=2)

# print(json.dumps(response, indent=2))
print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

completed at: Dec 19 2019 00:01:44


## Test 1. Sentiment related to Named Entities

The following test uses Watson for detecting sentiment related to target terms.

The target terms are user defined and here based on the spaCy NER model with the supplementary component.

The technique works by sending a text and list of target terms to the API and the scores related to each are returned.

These results show this technique negatively scores a number of entities which would reasonably be expected to be positive.

In [425]:
with open(os.path.join(filepath, "bush_filelist.json"), 'r') as fp:
    bush_filelist = json.load(fp)
    
text = ''
    
for file in bush_filelist[3:]:
    with open(bushpath + file[1], 'r') as fp:
        text = text + fp.read()
        
#doc = nlp(text)

## create a list of named entities for analysis and store results in Bush_Analysis_v2.json or BinLaden_Analysis_v2.json

targets = list({ent.lower_ for ent in doc.ents if ent.label_ in named_entities})

# response = service.analyze(text=text, features=Features( \
#                                                         sentiment=SentimentOptions(targets=targets), \
#                                                         entities=EntitiesOptions(sentiment=True), \
#                                                         keywords=KeywordsOptions(sentiment=True,emotion=True)
#                                                       )).get_result()

with open(os.path.join(resultspath, "Bush_Analysis_v2.json"), "wb") as f:
    f.write(json.dumps(response).encode("utf-8"))

print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

completed at: Dec 19 2019 01:06:18


### TextBlob

(insert information about TextBlob)

In [ ]:
from textblob import TextBlob
from textblob.np_extractors import ConllExtractor
from textblob.sentiments import NaiveBayesAnalyzer
extractor = ConllExtractor()

In [ ]:
with open(r'C:\Users\Steve\Documents\CulturalViolence\George Bush\20010120-Address to Joint Session of Congress Following 911 Attacks.txt', 'r') as fp:
    doc = nlp(fp.read())
    
for sentence in doc.sents:
    displacy.render(sentence, style = "ent")

In [794]:
with open(os.path.join(filepath, "Bush_Analysis_v2.json", 'r') as fp:
    response = json.load(fp)
    
table = response["sentiment"]["targets"]
keys = "sentiment"
frames = []

print(f'document sentiment score:{response["sentiment"]["document"]["score"]}')
print(f'document sentiment: {response["sentiment"]["document"]["label"]}')

sentiment = "negative"
frames = []
for entry in table:
      
      objs = {"sentiment" : entry["score"], "label" : entry["label"]}
      
      if objs["label"] == sentiment:
          df = pd.DataFrame(objs, index = [entry["text"]], columns = list(objs.keys())).fillna("")
          frames.append(df)
    
display(pd.concat(frames, sort = False).sort_values("sentiment", ascending = True).style.background_gradient(cmap=cmp))

document sentiment score:-0.295518
document sentiment: negative


,sentiment,label
muslim friends,-0.932519,negative
arab friends,-0.932519,negative
pakistanis,-0.898932,negative
el salvador,-0.898932,negative
iran,-0.898932,negative
israelis,-0.898932,negative
japan,-0.898932,negative
india,-0.898932,negative
american citizens,-0.898815,negative
gambia,-0.885745,negative


### Sentiment Scores for Watson Defined Entities

This final test looks at the scores asigned to the named entities identified by Watson as opposed to those which are user defined.

In the first instance, while Al Qaeda and the Taliban are identified, the Egyptian Islamic Jihad" and "Islamic Movement of Uzbekistan" - both identified by Bush as adversaries - have not been identified by Watson.

Much like the other Watson components, entities which would reasonably be expected to be scored positively are scored negatively.

In [861]:
table = response["entities"]

frames = []

print(f'document sentiment score:{response["sentiment"]["document"]["score"]}')
print(f'document sentiment: {response["sentiment"]["document"]["label"]}')

#text => sentiment => relvance => emotion => count

#print([e["text"] for e in table])      

sentiment = "negative"

print(len(table))
for entry in table:
      
      objs = {'count' : entry["count"], 'type' : entry["type"], 'sentiment': entry["sentiment"]["score"], 'label' : entry["sentiment"]["label"]}
      
      if objs["label"] == sentiment:
          df = pd.DataFrame(objs, index = [entry["text"]], columns = list(objs.keys())).fillna("")

          frames.append(df)
cmp = "Reds"
### don't work on this one
display(pd.concat(frames, sort = False).sort_values(["sentiment"], ascending = False).style.background_gradient(cmap=cmp))

document sentiment score:-0.295518
document sentiment: negative
50


,count,type,sentiment,label
Congress,9,Organization,-0.314552,negative
New York,4,Location,-0.340878,negative
USA,53,Location,-0.377997,negative
Cairo,1,Location,-0.440659,negative
Pennsylvania,3,Location,-0.462063,negative
Asia,4,Location,-0.514639,negative
Afghanistan,13,Location,-0.518789,negative
Brandenburg Gate,1,Facility,-0.57228,negative
Seoul,1,Location,-0.57228,negative
Taliban,14,Organization,-0.609357,negative


## Test 2. Sentiment Scores for Seed Words of the Group Schema

This next test shows how Watson scores feature terms of the text.

The feature terms are assigned by the Watson API.

This test develops upon a simple score and provides some degree of explanation as to why a feature is scored negatively.

For example:
- Al Qaeda is scored negatively along with the emotion of fear.
- The Taliban Regime is scored negatively with the emotion of disgust.
- Terrorists is scored negatively with the emotion of anger.
- American people is also scored negatively but with the emotion of sadness.

While these results seem plausible, however, the API does not seem to provide much explanatory value for the other features.

Moreover, there is no explanation as to why these decisions have been made.

In [876]:
table = response["keywords"]

frames = []

print(f'document sentiment score:{response["sentiment"]["document"]["score"]}')
print(f'document sentiment: {response["sentiment"]["document"]["label"]}')

#text => sentiment => relvance => emotion => count

sentiment = "negative"

for entry in table:

      objs = {"count" : entry["count"], "sentiment" : entry["sentiment"]["score"], "label" : entry["sentiment"]["label"], \
             "sadness" : entry["emotion"]["sadness"], "joy" : entry["emotion"]["joy"], "fear" : entry["emotion"]["fear"], \
             "disgust": entry["emotion"]["disgust"], "anger" : entry["emotion"]["anger"]}
      
      if objs["label"] == sentiment:

          df = pd.DataFrame(objs, index = [entry["text"]], columns = list(objs.keys())).fillna("")

          frames.append(df)
      
print(len(table))
cmp = "Reds"
### don't work on this one
display(pd.concat(frames, sort = False).sort_values("sentiment", ascending = True).style.background_gradient(cmap=cmp))

document sentiment score:-0.295518
document sentiment: negative
50


,count,sentiment,label,sadness,joy,fear,disgust,anger
name of Allah,2,-0.93206,negative,0.083546,0.417668,0.178262,0.158497,0.106093
terrorists' directive commands,1,-0.929267,negative,0.227984,0.005496,0.33787,0.459285,0.376797
al Qaeda,6,-0.871133,negative,0.189497,0.171494,0.617199,0.16837,0.142764
end lives,1,-0.869587,negative,0.427849,0.31318,0.201269,0.083121,0.146949
own nations,1,-0.855685,negative,0.261733,0.113977,0.365655,0.19303,0.194928
surprise attacks,1,-0.838343,negative,0.130122,0.053701,0.387704,0.548293,0.112611
Latin America,2,-0.801347,negative,0.604108,0.112159,0.123533,0.311312,0.076314
act of war,1,-0.742941,negative,0.442012,0.025592,0.280145,0.06107,0.347208
Army general,1,-0.715533,negative,0.235712,0.052283,0.115928,0.279271,0.2507
forgetting September,1,-0.680858,negative,0.411949,0.087484,0.110282,0.177939,0.049822


## Test 3: using IBM Watson and Text Blob for Detecting Elevation and Debasement Statements

This test is based on the following steps

1. take sentences containing named entity and mode of influence
2. sentiment analysis using some library
3. if the sentence is positive entity is tagged as elevation
4. if the sentence is negative entity is tagged as debasement


In [ ]:

    
#list of named entities
named_entities = set()
named_entities = {"NORP", "GPE", "NORP", "PERSON", "ORG"}

#list of labels for each mode of influence
labels = set()
for value in group_typology.values():
    for subcat in value.values():
        for term in list(subcat.keys()):
            labels.add(term)

# a dictionary object of ingroup and outgroup sentences determined by sentiment score
groups = {"ingroup" : [], "outgroup" : []}

for sentence in tqdm.tqdm(doc.sents, total = len(list(doc.sents))):
    
    #get the tokens.ent_types_ for each token in the sentence
    ent_types = {token.ent_type_ for token in sentence if token.ent_type_}
    
    # if the sentence ent_types contain both a named entities and mode of influence
    if not ent_types.isdisjoint(named_entities) and not ent_types.isdisjoint(labels):
        
        # get the sentiment score for TextBlob
        textblob_score = TextBlob(sentence.text).sentiment.polarity
        
        # get the score for IBM Watson
        #response = service.analyze(text=sentence.text,features=Features(sentiment=SentimentOptions())).get_result()
        #watson_score = response['sentiment']['document']['score']
        watson_score = 0
        
        # result for the sentence
        result = (sentence.start, sentence.end, textblob_score, watson_score)
        
        ## append to ingroup category if either have a positive score and vice versa
        if watson_score > 0 or textblob_score > 0:
            groups["ingroup"].append(result)
        elif textblob_score < 0 or watson_score < 0:
            groups["outgroup"].append(result)
        
with open(r"C:\Users\Steve\Documents\CulturalViolence\KnowledgeBases\group_sentiment_ruler.json", "wb") as f:
    f.write(json.dumps(groups).encode("utf-8"))

print(f'completed at: {datetime.datetime.now().strftime("%b %d %Y %H:%M:%S")}')

# Test Results

We can see that neither is able to detect the sentences defining the ingroup and outgroup of a text.

Despite Watson's sophistication, it provides no more value than TextBlob, which is a much simpler technology.

Through observation, we know the following are outgroup named entities from Bush's speech:
1. al Qaeda
2. Egyptian Islamic Jihad
3. Islamic Movement of Uzbekistan
4. the Taliban

A successful test with return these named entities as an outgroup.

In [872]:
with open(os.path.join(resultspath, "group_sentiment_namedconcept.json"), 'r') as fp:
    groups = json.load(fp)

gradient = "elevation"
for index in groups[group]:
    print('-----')
    print(f'these are the result for {gradient}')
    
    sentence = doc[index[0]:index[1]]
    
    token = []
    pos = []
    ent_type = []
    sentiment = []
    dep = []
    
    sent_table = {
        "token" : [token.text for token in sentence],
        "dep" : [token.dep_ for token in sentence],
        "ent_type" : [token.ent_type_ for token in sentence],
        "pos" : [token.pos_ for token in sentence],
        "sentiment" : [TextBlob(token.text).sentiment.polarity for token in sentence]
    }

    print(f'{gradient} sentence sentiment score for TextBlob: {index[2]}')
    print(f'{gradient} sentence sentiment score for Watson: {index[3]}')
    
    display(pd.DataFrame.from_dict(sent_table, orient = "index"))

-----
these are the result for ingroup
ingroup sentence sentiment score for TextBlob: 0.6666666666666666
ingroup sentence sentiment score for Watson: -0.670821


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26
token,We,have,seen,it,in,the,courage,of,passengers,",",who,rushed,terrorists,to,save,others,on,the,ground,--,passengers,like,an,exceptional man,named,Todd Beamer,.
dep,nsubj,aux,ROOT,dobj,prep,det,pobj,prep,pobj,punct,nsubj,relcl,dobj,aux,xcomp,dobj,prep,det,pobj,punct,dobj,prep,det,pobj,acl,oprd,punct
ent_type,,,,,,,BENEVOLANCE,,ECONGROUP,,,,CRIMEGROUP,,,,,,LOCATION,,ECONGROUP,,,SOCGROUP,,PERSON,
pos,PRON,VERB,VERB,PRON,ADP,DET,NOUN,ADP,NOUN,PUNCT,PRON,VERB,NOUN,PART,VERB,NOUN,ADP,DET,NOUN,PUNCT,NOUN,ADP,DET,ADJ,VERB,PROPN,PUNCT
sentiment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.666667,0,0,0


-----
these are the result for ingroup
ingroup sentence sentiment score for TextBlob: 0.8
ingroup sentence sentiment score for Watson: 0.643663


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
token,And,would,you,please,help,me,to,welcome,his,wife,",",Lisa Beamer,",",here,tonight,.
dep,cc,aux,nsubj,intj,ROOT,dobj,aux,xcomp,poss,dobj,punct,appos,punct,advmod,npadvmod,punct
ent_type,,,,,,,,,,FAMILY,,PERSON,,,TIME,
pos,CCONJ,VERB,PRON,INTJ,VERB,PRON,PART,VERB,DET,NOUN,PUNCT,PROPN,PUNCT,ADV,NOUN,PUNCT
sentiment,0,0,0,0,0,0,0,0.8,0,0,0,0,0,0,0,0


-----
these are the result for ingroup
ingroup sentence sentiment score for TextBlob: 0.2
ingroup sentence sentiment score for Watson: 0.957452


...

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32
token,As,long,as,the United States of America,is,determined,and,strong,",",this,will,not,be,an,age,of,terror,;,this,will,be,an,age,of,liberty,",",here,and,across,the,world,.,\n
dep,advmod,advmod,mark,nsubj,advcl,acomp,cc,conj,punct,nsubj,aux,neg,ccomp,det,attr,prep,pobj,punct,nsubj,aux,ROOT,det,attr,prep,pobj,punct,advmod,cc,conj,det,pobj,punct,
ent_type,,,,GPE,,,,BENEVOLANCE,,,,,,,,,MALEVOLANCE,,,,,,,,GPEIDEOLOGY,,,,,,LOC,,
pos,ADV,ADV,ADP,DET,VERB,ADJ,CCONJ,ADJ,PUNCT,DET,VERB,ADV,VERB,DET,NOUN,ADP,NOUN,PUNCT,DET,VERB,VERB,DET,NOUN,ADP,NOUN,PUNCT,ADV,CCONJ,ADP,DET,NOUN,PUNCT,SPACE
sentiment,0,-0.05,0,0,0,0,0,0.433333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


-----
these are the result for ingroup
ingroup sentence sentiment score for TextBlob: 0.8
ingroup sentence sentiment score for Watson: 0.806233


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
token,It,was,given,to,me,by,his,mom,",",Arlene,",",as,a,proud memorial,to,her,son,.
dep,nsubjpass,auxpass,ROOT,dative,pobj,agent,poss,pobj,punct,appos,punct,prep,det,pobj,prep,poss,pobj,punct
ent_type,,,,,,,,FAMILY,,PERSON,,,,RELENTITY,,,FAMILY,
pos,PRON,VERB,VERB,ADP,PRON,ADP,DET,NOUN,PUNCT,PROPN,PUNCT,ADP,DET,ADJ,ADP,DET,NOUN,PUNCT
sentiment,0,0,0,0,0,0,0,0,0,0,0,0,0,0.8,0,0,0,0


-----
these are the result for ingroup
ingroup sentence sentiment score for TextBlob: 0.0
ingroup sentence sentiment score for Watson: 0.691251


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
token,In,all,that,lies,before,us,",",may,God,grant,us,wisdom,",",and,may,He,watch,over,the United States of America,.,\n
dep,prep,pobj,nsubj,relcl,prep,pobj,punct,aux,nsubj,ROOT,dative,dobj,punct,cc,aux,nsubj,conj,prep,pobj,punct,
ent_type,,,,CHASTISEMENT,,,,,DEITY,,,,,,,,,,GPE,,
pos,ADP,DET,DET,VERB,ADP,PRON,PUNCT,VERB,PROPN,VERB,PRON,NOUN,PUNCT,CCONJ,VERB,PRON,VERB,ADP,DET,PUNCT,SPACE
sentiment,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
